LangChain Tool
```
Tool : LLM 에이전트가 수행할 수 있는 외부기능이나 api를 말함(날씨정보나 웹검색, 계산기 등)
이유 : gpt는 최신 정보에 접근할 수 없다.
구조 : 이름 설명 함수실행로직
툴 호출 방식 : GPT-4 모델들은 OpenAI의 함수호출 기능을 통해서 툴을 직접 호출할 수 있다.
```

외부 API 호출 Tool 구현(날씨API, 뉴스API)

In [2]:
%pip install langchain-core langchain-openai openai python-dotenv requests

Note: you may need to restart the kernel to use updated packages.


In [1]:
# .env 로드
from dotenv import load_dotenv
import os
load_dotenv()

print(os.environ['OPEN_WEATHER_MAP'][:5])
print(os.environ['NEWSAPI_KEY'][:5])

5783b
43ec2


날씨정보 조회

In [11]:
from pydantic import BaseModel, Field

# 입력 스키마
class WeatherInput(BaseModel):
    city: str = Field(description = "날씨를 조회할 도시 이름(영문)")
    
# 날씨정보 조회 함수
def get_weather(city: str) -> str:
    import requests
    api_key = os.getenv("OPEN_WEATHER_MAP")
    url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q":city,
        "appid":api_key,
        "units": "metric", #섭씨온도
        "lang": "kr"
    }
    response = requests.get(url,params=params)
    data = response.json()
    if data.get('cod') != 200:
        return "날씨 정보를 가져올 수 없습니다."
    temp = data['main']['temp']
    desc = data['weather'][0]['description']
    return f"{city}의 현재 기온은 {temp}도이며, 날씨는 {desc}입니다."

In [ ]:
#pydantic 모델을 사용하여 입력값 검증
class WeatherInput(BaseModel):
    city: str = Field(description = "날씨를 조회할 도시 이름(영문)")

class MyClass:
    def __init__(self, ab:str):
        self.city = ab

MyClass(ab=100) # 사용자가 의도한 문자열이 전달되지 않아도 python은 타입에러를 발생시키지 않음
result = WeatherInput(city=100) #문자열이 아니면 Typeerror를 발생시킴. pydantic 모델을 사용했기 때문(validation check를 위해서)
# result = WeatherInput(city='Seoul') #이렇게 문자열로 넣어야만 작동함
result.city

'Seoul'

In [3]:
get_weather('Seoul')

'Seoul의 현재 기온은 22.76도이며, 날씨는 연무입니다.'

뉴스검색

In [14]:
from pydantic import BaseModel, Field

# 입력 스키마
class NewsInput(BaseModel):
    keyword: str = Field(description = "뉴스를 조회할 키워드")

def get_news(keyword:str) -> str:
    import requests
    api_key = os.getenv("NEWSAPI_KEY")
    url = "https://newsapi.org/v2/everything"
    params = {
        "q" : keyword,
        "apiKey" : api_key,
        "language" : "ko",
        "sortBy" : "publishedAt", #최신뉴스
        "pageSize" : 1 #최근 1건
    }
    response = requests.get(url,params=params)
    data = response.json()
    articles = data.get('articles')
    if not articles:
        return "해당 키워드에 대한 뉴스가 없습니다"
    # 가장 첫번째 뉴스 선택
    top_news = articles[0]
    title = top_news.get('title', '제목 없음')
    source = top_news.get('source', {}).get('name', '출처 없음')
    return f'{keyword}에 대한 최신 뉴스: "{title}" (출처: {source})'

In [5]:
print(get_news('삼성전자'))

삼성전자에 대한 최신 뉴스: "삼성전자, 6월까지 안드로이드 15 기반 '원UI 7' 업데이트 마무리" (출처: Kbench.com)


LangChain Tool 객체로 변환
```
get_weather, get_news 함수를 Tool로 등록
```

In [15]:
from langchain_core.runnables import RunnableLambda
# 날씨함수 runnable -> tool 변환
weather_runnable = RunnableLambda(lambda x: get_weather(x['city']))
weather_tool = weather_runnable.as_tool(
    name = "get_weather",
    description="도시의 현재 날씨 정보를 반환합니다.",
    # arg_types={"city":str}
    args_schema= WeatherInput
)
news_runnable = RunnableLambda(lambda x: get_news(x['keyword']))
news_tool = news_runnable.as_tool(
    name = "get_news",
    description="키워드에 대한 최신 뉴스 정보를 반환합니다",
    # arg_types={"keyword":str}
    args_schema= NewsInput
)
print(f'tool 이름:{weather_tool.name} 설명: {weather_tool.description}')
print(f'tool 이름:{news_tool.name} 설명: {news_tool.description}')

tool 이름:get_weather 설명: 도시의 현재 날씨 정보를 반환합니다.
tool 이름:get_news 설명: 키워드에 대한 최신 뉴스 정보를 반환합니다


In [16]:
# 툴 없이
from langchain_openai import ChatOpenAI
# 프롬프트 템플릿
from langchain_core.prompts import ChatPromptTemplate
# 출력 파서
from langchain_core.output_parsers import StrOutputParser
user_question = '서울 날씨 알려줘'
# 사용자 질문을 처리하는 프롬프트 템플릿
user_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 날씨와 뉴스 정보를 제공하는 AI입니다."),
    ("human", "{question}")
])
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.0)
chain = user_prompt | llm
# 사용자 질문을 처리하는 체인 실행
result = chain.invoke({"question": user_question})
print(result)

content='현재 서울의 날씨는 실시간으로 확인할 수 없지만, 일반적으로 10월의 서울은 가을 날씨로 쌀쌀하고 맑은 날이 많습니다. 평균 기온은 약 10도에서 20도 사이입니다. 구체적인 날씨 정보는 기상청 웹사이트나 날씨 앱을 통해 확인하시기 바랍니다. 추가로 궁금한 점이 있으면 말씀해 주세요!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 29, 'total_tokens': 121, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BbfBFGdXssSNhHMfY05AbzOqa0X69', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--f048f132-85a5-466a-bf29-136eba7e5546-0' usage_metadata={'input_tokens': 29, 'output_tokens': 92, 'total_tokens': 121, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


LLM 최종 답변 생성 요청

In [18]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage
# 프롬프트 구성: 시스템메시지 + 사용자 질문 + 결과를 맥락으로 제공
weather_info = get_weather('seoul')
system_prompt = "당신은 유용한 AI 어시스턴트입니다. 사용자 질문에 맞게 제공된 정보를 활용해 답변하세요"
tool_info_prompt = f'도구가 제공한 추가 정보:{weather_info}'
ask_prompt = '위 정보를 참고해서 사용자 질문에 답변하세요.'
#system message랑 Human은 고정, AI가 그거에 따라 답변해주고, 그 다음 Human이 답변함
#실제정보가 AI가 한 것처럼 change 해줄것..?
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_question),
    AIMessage(content=tool_info_prompt), # 도구 결과를 마치 ai의 답변처럼 추가
    HumanMessage(content=ask_prompt)
]
result = llm(messages)  # LLM에 메시지 전달
print(result.content)

/var/folders/9c/x8552y5x1ks7f22561j09f500000gn/T/ipykernel_50391/1587631345.py:15: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm(messages)  # LLM에 메시지 전달


현재 서울의 날씨는 연무이며, 기온은 약 24.76도입니다. 외출 시 가벼운 옷차림이 적합할 것 같습니다. 추가적인 날씨 정보가 필요하시면 말씀해 주세요!


여러 tool을 연결하도록 에이전트 구성

In [ ]:
# 에이전트를 생성하고 등록
from langchain.agents import initialize_agent, AgentType
# 앞에서 만든 tool 객체 리스트를 준비
tools = [weather_tool, news_tool]